Setup required enviroment, it may take a few minutes

In [ ]:
%%bash
#!/bin/bash

KERNEL_NAME="Muon-HAT-2020"

set -e

# Download miniconda3 install script
if [[ ! -f $HOME/miniconda3.sh ]]; then
    wget --progress=dot:giga https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O $HOME/miniconda3.sh
fi

# Install miniconda3
if [[ ! -d $HOME/miniconda3 ]]; then
    bash $HOME/miniconda3.sh -b -f -u -p $HOME/miniconda3
    source $HOME/miniconda3/etc/profile.d/conda.sh
else
    source $HOME/miniconda3/etc/profile.d/conda.sh
fi

# Create conda environment
if [[ ! -d $HOME/miniconda3/envs/$KERNEL_NAME ]]; then
    conda env create -f environment.yml --name $KERNEL_NAME
fi
conda activate $KERNEL_NAME

# Make a wrapper script to load miniconda python
if [[ ! -f python_wrapper.sh ]]; then
cat << 'EOF' > python_wrapper.sh
#!/bin/bash
source $HOME/miniconda3/etc/profile.d/conda.sh
conda activate Muon-HAT-2020
exec python3 "$@"
EOF
chmod +x python_wrapper.sh
fi

# Create the kernel
if [[ ! -d $HOME/.local/share/jupyter/kernels/$KERNEL_NAME ]]; then
mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python3", 
 "argv": [
  "$PWD/python_wrapper.sh", 
  "-m", 
  "ipykernel_launcher", 
  "-f", 
  "{connection_file}"
 ]
}
EOF
fi

# Report OK
echo "Loaded $KERNEL_NAME!"